# 构建神经网络

神经网络由对数据执行操作的层/模块组成。 `torch.nn` 命名空间提供了构建自己的神经网络所需的所有构件。PyTorch 中的每个模块都子类继承 `nn.Module`。神经网络本身就是一个由其他模块（层）组成的模块。这种嵌套结构可以轻松构建和管理复杂的架构。在下面的章节中，我们将构建一个神经网络，对 FashionMNIST 数据集中的图像进行分类。

In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

## 获取训练的设备

我们希望能在 GPU 或 MPS 等硬件加速器（如果可用）上训练我们的模型。让我们检查一下 `torch.cuda` 或 `torch.backends.mps` 是否可用，否则我们就使用 CPU。

In [3]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


[ROCT-Thunk-Interface/src/hymgr.c:309, WARNING]: Version mismatch


## 定义类

我们通过子类继承 `nn.Module` 来定义神经网络，并在 `__init__` 中初始化神经网络层。每个 `nn.Module` 子类都在 `forward` 方法中实现对输入数据的操作。

In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

我们创建一个 `NeuralNetwork` 实例，将其移动到设备上并打印其结构。

In [5]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


要使用模型，我们需要将输入数据传递给它。这将执行模型的 `forward` 以及一些后台操作。不要直接调用 `model.forward()`！

在输入上调用模型会返回一个二维张量，其中 `dim=0` 对应于每个类的 10 个原始预测值中的每个输出，`dim=1` 对应于每个输出的单独值。我们通过 `nn.Softmax` 模块的实例来获取预测概率。

In [6]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([4], device='cuda:0')


## 模型层

让我们来分解一下 FashionMNIST 模型的各个层。为了说明这一点，我们将以 3 幅大小为 28x28 的图像为样本，看看在通过网络时会发生什么。

In [7]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


### `nn.Flatten`

我们对 `nn.Flatten` 层进行初始化，将每幅 28x28 的二维图像转换成一个包含 784 个像素值的连续数组（保持最小批次维度（dim=0））。

In [8]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


### `nn.Linear`

线性层是一个利用其存储的**权重**和**偏置**对输入进行线性变换的模块。

In [9]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


### `nn.ReLU`

非线性激活可以在模型的输入和输出之间建立复杂的映射关系。它们应用于线性变换之后，以引入非线性，帮助神经网络学习各种现象。

在这个模型中，我们在线性层之间使用了 `nn.ReLU`，但还有其他激活方式可以在模型中引入非线性。

In [10]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.1658, -0.1091,  0.7256,  0.0902, -0.3689,  0.1786,  0.5108, -0.4763,
          0.1220,  0.2876,  0.0839,  0.2953,  0.3428, -0.5491,  0.6158,  0.0958,
         -0.5334,  0.5270, -0.3253,  0.3302],
        [ 0.0821,  0.0367,  0.8428,  0.1197, -0.2776, -0.3507,  0.1103, -0.1259,
         -0.0427,  0.2837,  0.2168,  0.1771,  0.5137, -0.4644,  0.6042,  0.1068,
         -0.3798,  0.6505, -0.3315,  0.2943],
        [ 0.2987,  0.0610,  0.2093,  0.2466, -0.7297, -0.0776, -0.0964, -0.1722,
         -0.1917,  0.2947,  0.0055,  0.0557,  0.1988, -0.4674,  0.3372,  0.2811,
         -0.5890,  0.8371, -0.4154,  0.2451]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.1658, 0.0000, 0.7256, 0.0902, 0.0000, 0.1786, 0.5108, 0.0000, 0.1220,
         0.2876, 0.0839, 0.2953, 0.3428, 0.0000, 0.6158, 0.0958, 0.0000, 0.5270,
         0.0000, 0.3302],
        [0.0821, 0.0367, 0.8428, 0.1197, 0.0000, 0.0000, 0.1103, 0.0000, 0.0000,
         0.2837, 0.2168, 0.1771, 0.5137, 0.0000, 0.60

### `nn.Sequential`

`nn.Sequential` 是一个有序的模块容器。数据按照定义的**相同顺序**通过所有模块。您可以使用顺序容器来组建一个快速网络，如 `seq_modules`。

In [11]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

### `nn.Softmax`

神经网络的最后一层线性层返回 logits（原始值，单位为 [-infty，infty] ），并将其传递给 `nn.Softmax` 模块。`dim` 参数表示数值必须相加为 1 的维度。

In [12]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

## 模型参数

神经网络中的许多层都是参数化的，即在训练过程中优化相关的权重和偏置。子类化 `nn.Module` 会自动跟踪模型对象内定义的所有字段，并使用模型的 `parameters()` 或 `named_parameters()` 方法访问所有参数。

在本例中，我们遍历每个参数，并打印其大小和数值预览。

In [13]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0190, -0.0101, -0.0104,  ...,  0.0207,  0.0292,  0.0155],
        [ 0.0323, -0.0250, -0.0064,  ...,  0.0035, -0.0086,  0.0038]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0089, -0.0057], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0359,  0.0021, -0.0428,  ...,  0.0146, -0.0092,  0.0088],
        [-0.0234, -0.0034,  0.0125,  ..., -0.0109,  0.0309,  0.0173]],
       device='cuda:0', grad_fn=<Sl

## Further Reading

+ [`torch.nn API`](https://pytorch.org/docs/stable/nn.html)